In [ ]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_ENABLE_ONEDNN_OPTS'] =  "0"



import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy

from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.datasets import mnist

## Librerías
import seaborn as sns
import sys
import cv2 as cv2
import glob
from PIL import Image
import tensorflow as tf
import sys
import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import gc
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
import os


from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler

print(tf.__version__)

In [ ]:
## Uso de GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

# Dataset

In [3]:
## function to load images
def load_binary_images(file_names):  
    image = np.load(file_names).astype(np.float32)
    image = np.expand_dims(image, axis=-1)
    image = tf.convert_to_tensor(image, dtype=tf.float32)
    image = tf.image.resize(image, [32,32])   
    ## binariazar
    image = tf.where(image > 0.5, 1.0, 0.0)  
    return image

def load_images(file_names):
    image = np.load(file_names).astype(np.float32)
    #expand_dims
    image = np.expand_dims(image, axis=-1)
    image = tf.convert_to_tensor(image, dtype=tf.float32)
    image = tf.image.resize(image, [32,32])/255.0    
      
    return image

def min_max_scaler(ruta,image):
    ## load images in the path
    images = glob.glob(ruta + '/*.npy')
    images = sorted(images)
    ## load images
    images = [load_images(image) for image in images]
    ## min max scaler of thei image using the max and min of all images
    images = np.array(images)
    max_value = np.max(images)
    min_value = np.min(images)
    image = tf.math.abs((image - min_value)/(max_value - min_value))
    return image   

## Function to get contours and features
def get_contours_and_features(binary_map):
    #https://docs.opencv.org/4.x/d3/d05/tutorial_py_table_of_contents_contours.html
    #binary_map = cv2.cvtColor(binary_map, cv2.COLOR_BGR2GRAY)
    contours, hierarchy = cv2.findContours(binary_map, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours_features = []
    for contour in contours:
        error = 1e-5
        moments = cv2.moments(contour)
        cx = moments['m10'] / (moments['m00'] + error)
        cy = moments['m01'] / (moments['m00'] + error)
        center_of_mass = [cx, cy]
        x,y,w,h = cv2.boundingRect(contour)        
        rect_area = w*h
        features = {
            'bounding_box': (x,y,w,h),
            'area': cv2.contourArea(contour),
            'perimeter': cv2.arcLength(contour, True),       
            #'solidity': np.float32(cv2.contourArea(contour))/cv2.convexHull(contour),
            'equivalent_diameter': np.sqrt(4*cv2.contourArea(contour)/np.pi),            
            'moments': moments,
            'center_of_mass': center_of_mass,
            'contour': contour
        }
        contours_features.append(features)
        del features
    #plt.imshow(contours_map, cmap='gray')
    return contours_features

# function to get a determined property from a list of contours features (area by default)
def get_item(contour_features, key='area'):
    areas = []
    for contour_feature in contour_features:
        area =  contour_feature[key]
        areas.append(area)
    return areas

## function to get geometric_attributes
def get_geometric_atributes(binary_images):
    descriptors = []
    for binary_img in binary_images:
        ## Formato
        image = binary_img.numpy().astype(np.uint8)       
        
        ## Capturar contornos
        contour_features = get_contours_and_features(image)
        
        ## Calcular vector de áreas de poro (todos los poros)
        areas = get_item(contour_features, key='area')

        ## Calcular vector de perímetros de poro (todo los poros)
        pmtro = get_item(contour_features, key='perimeter')

        ## Calcular el diametro equivalente de los poros
        eq_diameter = get_item(contour_features, key='equivalent_diameter')                

        descriptor = [np.mean(areas), np.mean(pmtro),np.mean(eq_diameter)]
        
        descriptors.append(descriptor)
        
    
    return descriptors

## function to get the middle image route in the folder
def get_middle_image(folder_route):
    ## get list of images
    images = glob.glob(folder_route + '/*.npy')
    ## order images
    images = sorted(images)
    ## get index of middle image
    if len(images) == 0:
        indx = -1
        middle_image = 'empty'
    else:
        indx = len(images)//2
        middle_image = images[indx]   
    
    return middle_image

In [ ]:
## annotations to get nodule_features
annotations_cvs_4R = pd.read_csv('/data/Datasets/Nodules_ISBI/meta_created_info_3d_3R.csv')
annotations_csv_3R = pd.read_csv('/data/Datasets/Nodules_ISBI/meta_created_info_3d_4R.csv')
annotations = pd.concat([annotations_cvs_4R, annotations_csv_3R], ignore_index=True)
## drop malignancy = 3
annotations = annotations[annotations['malignancy'] != 3]
annotations

In [ ]:
## ruta dataset
rutas_images_npy = glob.glob('/data/Datasets/Nodules_ISBI/images/*/*.npy')

## order list by folder
rutas_images_npy = sorted(rutas_images_npy)

## get list of folders
folders = [ruta.split('/')[-2] for ruta in rutas_images_npy]
folders = np.unique(folders)

## build dataframe
rutas_images = []
rutas_masks = []
areas = []
perimetros = []
diametros = []
calsifications = []
spiculations = []
lobulations = []
sphericities = []
textures = []
margins = []
subletys = []

## cancer or not
labels = []
    
for folder in folders:
    ruta = '/data/Datasets/Nodules_ISBI/images/' + folder
    ruta_masc = '/data/Datasets/Nodules_ISBI/masks/' + folder
    
    ## load middle image route
    image_route = get_middle_image(ruta)
    name_image = image_route.split('/')[-1]
    mask_route = ruta_masc + '/' + name_image       
    
    if image_route != 'empty':    
        ## get attributes
        image = load_images(image_route)
        ## min-max scaler
        image = min_max_scaler(ruta, image)
        ## binary image
        binary_image = load_binary_images(mask_route)   
        ## get features of the image            
        features = annotations[annotations['folder'] == folder]
        if features.shape[0] != 0:     
            ## append rutas
            geometric_attributes = get_geometric_atributes([binary_image])[0]
            rutas_images.append(image_route)
            rutas_masks.append(mask_route)
            
            ## append attributes
            areas.append(geometric_attributes[0])
            perimetros.append(geometric_attributes[1])
            diametros.append(geometric_attributes[2])    
            
            ## evaluar si son nan e iomprimir solo si los  
              
            malignancy = features['malignancy'].values[0]
            calsification = features['calcification'].values[0]
            spiculation = features['spiculation'].values[0]
            lobulation = features['lobulation'].values[0]
            texture = features['texture'].values[0]
            label = features['is_cancer'].values[0]
            margin = features['margin'].values[0]
            sublety = features['subtlety'].values[0]
            
            ## append featuresprint(features)
            calsifications.append(calsification)
            spiculations.append(spiculation)
            lobulations.append(lobulation)
            textures.append(texture)
            labels.append(label)
            margins.append(margin)
            subletys.append(sublety)


## build dataframe
df = pd.DataFrame()
df['ruta'] = rutas_images
df['mask'] = rutas_masks
df['area'] = areas
df['perimetro'] = perimetros
df['diametro'] = diametros
df['calsification'] = calsifications
df['spiculation'] = spiculations
df['lobulation'] = lobulations
df['texture'] = textures
df['margin'] = margins
df['sublety'] = subletys
df['label'] = labels

### eliminar filas con valores nan y decir cuantas se borraron
#print('Filas antes de eliminar nan:', df.shape[0])
#df = df.dropna()
#print('Filas después de eliminar nan:', df.shape[0])

df

## Dataset

In [ ]:
## shuffle data
df_train_1, df_test = train_test_split(df, test_size=0.2, random_state=42)

## validation split
val_pctg = 0.8
split_validation_index = int(df_train_1.shape[0]*val_pctg)
df_train = df_train_1.iloc[:split_validation_index]
df_validation = df_train_1.iloc[split_validation_index:]

print('---- Distribución de entrenamiento, validación y test para benignos y malignos ----')
print('Malignos: ')
print('Positive train shape: ', df_train[df_train['label'] == 'True'].shape[0])
print('Positive val shape: ', df_validation[df_validation['label'] == 'True'].shape[0])
print('Positive test shape: ', df_test[df_test['label'] == 'True'].shape[0])

print('Benignos: ')
print('Negative train shape: ', df_train[df_train['label'] == 'False'].shape[0])
print('Negative val shape: ', df_validation[df_validation['label'] == 'False'].shape[0])
print('Negative test shape: ', df_test[df_test['label'] == 'False'].shape[0])


## replace labels
df_train.loc[:, 'label'] = df_train['label'].replace({'True': 1, 'False': 0})
df_test.loc[:, 'label'] = df_test['label'].replace({'True': 1, 'False': 0})
df_validation.loc[:, 'label'] = df_validation['label'].replace({'True': 1, 'False': 0})

print('---- Dataset finales ----')
print('Train shape: ', df_train.shape[0])
print('Test shape: ', df_test.shape[0])
print('Validation shape: ', df_validation.shape[0])

In [ ]:
## plot 5 first images with mas on the df train dataset
plt.figure(figsize=(20,10))
for i in range(5):
    image = load_images(df_train['ruta'].values[i])    
    plt.subplot(1,10,i+1)
    plt.imshow(image, cmap='gray')   
    plt.title('Label: ' + str(df_train['label'].values[i]))

In [ ]:
## create dataset function 
features_to_use = ['area','lobulation', 'texture', 'spiculation', 'margin', 'calsification', 'sublety']


def create_train_dataset(batch,df):
    images_routes = df['ruta'].to_numpy()
    mask_routes = df['mask'].to_numpy()
    labels = df['label'].to_numpy()
    features = df[features_to_use].to_numpy()
    
    
    ## Data augmentation    
    def rotate90_image(image):
        ## rotate 90 degrees
        image = tf.image.rot90(image)
        return image
    
    def rotate180_image(arg):
        ## rotate 180 degrees
        image = tf.image.rot90(arg, k=2)
        return image
    
    def rotate270_image(arg):
        ## rotate 270 degrees
        image = tf.image.rot90(arg, k=3)
        return image
    
    images = []
    mask = []
    new_labels = []
    new_features = []
    for image_route, mask_route, label, features in zip(images_routes,mask_routes, labels,features):
        ## imagenes y mascaras originales
        image_original = load_images(image_route)
        mask_original = load_binary_images(mask_route)
        
        ## augmentaciones
        # imagen original
        #image_left_flip = flip_left_image(image_original)
        image_rotate90 = rotate90_image(image_original)
        image_rotate180 = rotate180_image(image_original)
        image_rotate270 = rotate270_image(image_original)
        #image_up_flip = flip_up_image(image_original)
        ## mascara original
        #mask_left_flip = flip_left_image(mask_original)
        mask_rotate90 = rotate90_image(mask_original)
        mask_rotate180 = rotate180_image(mask_original)
        mask_rotate270 = rotate270_image(mask_original)
        #mask_up_flip = flip_up_image(mask_original)
        
        ## append image augmentation
        images.append(image_original)        
        images.append(image_rotate90)
        images.append(image_rotate180)
        images.append(image_rotate270)
        
        
        ## append mask augmentation
        mask.append(mask_original)        
        mask.append(mask_rotate90)
        mask.append(mask_rotate180)
        mask.append(mask_rotate270)
        
        ## append labels        
        new_labels.append(label)
        new_labels.append(label)
        new_labels.append(label)
        new_labels.append(label)           
        
        ## append features  
        new_features.append(features)
        new_features.append(features)
        new_features.append(features)
        new_features.append(features)
        
        
        
        
    ## convert to tensor
    images = tf.convert_to_tensor(images, dtype=tf.float32)
    masks = tf.convert_to_tensor(mask, dtype=tf.float32)
    labels = tf.convert_to_tensor(new_labels, dtype=tf.float32)
    features = tf.convert_to_tensor(new_features, dtype=tf.float32)    
    
    
    dataset = tf.data.Dataset.from_tensor_slices((images, masks, features, labels))
    
    ## shuffle data
    dataset = dataset.shuffle(buffer_size=1000,seed=42)
    
    ## apply preprocess_image function
    dataset = dataset.map(lambda image, mask, features, label: (image, mask, features, label))
    
    dataset = dataset.batch(batch)
    
    return dataset


## create dataset function 
def create_test_dataset(batch,df):       
    images_routes = df['ruta'].to_numpy()
    mask_routes = df['mask'].to_numpy()
    labels = df['label'].to_numpy()
    features = df[features_to_use].to_numpy()
    
    
    images = []
    for image_route in images_routes:
        image = load_images(image_route)
        images.append(image)
        
    masks = []
    for mask_route in mask_routes:
        mask = load_binary_images(mask_route)
        masks.append(mask)
        
    ## convert to tensor
    images = tf.convert_to_tensor(images, dtype=tf.float32)
    masks = tf.convert_to_tensor(mask, dtype=tf.float32)
    labels = tf.convert_to_tensor(labels, dtype=tf.float32)
    features = tf.convert_to_tensor(features, dtype=tf.float32)    
    
    dataset = tf.data.Dataset.from_tensor_slices((images, masks, features, labels))
    
    ## apply preprocess_image function
    dataset = dataset.map(lambda image, masks, features, label: (image, masks, features, label))
      
    dataset.batch(batch)
    
    return dataset

## both-clases
train_dataset = create_train_dataset(8, df_train)


for image, mask, features, label in train_dataset:
    print(image.shape)
    print(mask.shape)
    print(features.shape)
    print(label.shape)    
    break

i = 0 
for image, mask, features, label in train_dataset:
    i += np.shape(image)[0]   

print('Total de imagenes en el dataset de entrenamiento:', i)


In [ ]:
for image, mask, features, label in train_dataset:
    ## plot 8 batch images and their mask
    plt.figure(figsize=(20,20))
    for i in range(8):
        plt.subplot(8,2,2*i+1)
        plt.imshow(image[i], cmap='gray')       
        plt.title('Image')
        plt.subplot(8,2,2*i+2)
        plt.imshow(mask[i], cmap='gray')
        plt.title('Mask')
        
    plt.tight_layout()
    plt.show()
    break

# Unet

In [ ]:
## decoder
nfilters = 16
latent_dim = 16
nx , ny = 32, 32


encoder_input = keras.Input(shape=(nx,ny,1), name='original_img')

c1 = layers.Conv2D(nfilters*1, kernel_size=(3,3),
               kernel_initializer='he_normal', padding='same')(encoder_input)
c1 = layers.BatchNormalization()(c1)
c1 = layers.Activation('relu')(c1)
p1 = layers.MaxPooling2D((2,2))(c1)
p1 = layers.Dropout(0.1)(p1)

c2 = layers.Conv2D(nfilters*2, kernel_size=(3,3),
               kernel_initializer='he_normal', padding='same')(p1)
c2 = layers.BatchNormalization()(c2)
c2 = layers.Activation('relu')(c2)
p2 = layers.MaxPooling2D((2,2))(c2)

c3 = layers.Conv2D(nfilters*4, kernel_size=(3,3),
               kernel_initializer='he_normal', padding='same')(p2)
c3 = layers.BatchNormalization()(c3)
c3 = layers.Activation('relu')(c3)
p3 = layers.MaxPooling2D((2,2))(c3)
p3 = layers.Dropout(0.1)(p3)

c4 = layers.Conv2D(nfilters*8, kernel_size=(3,3),
               kernel_initializer='he_normal', padding='same')(p3)
c4 = layers.BatchNormalization()(c4)
c4 = layers.Activation('relu')(c4)
p4 = layers.MaxPooling2D((2,2))(c4)
p4 = layers.Dropout(0.1)(p4)

c5 = layers.Conv2D(nfilters*16, kernel_size=(3,3),
               kernel_initializer='he_normal', padding='same')(p4)

# Flatten and Embedding
c5_flatten = layers.Flatten()(c5)
embbedding = layers.Dense(latent_dim, name='embedding')(c5_flatten)

encoder = keras.Model(encoder_input, [embbedding , [c1,c2,c3,c4]], name='encoder')
encoder.summary()

In [ ]:
## decoder 
decoder_input = keras.Input(shape=(latent_dim,), name='encoded_img')
c1_dec = keras.Input(shape=c1.shape[1:], name='c1_input')
c2_dec = keras.Input(shape=c2.shape[1:], name='c2_input')
c3_dec = keras.Input(shape=c3.shape[1:], name='c3_input')
c4_dec = keras.Input(shape=c4.shape[1:], name='c4_input')

## decoder layers from embedding
d1_dec = layers.Dense(2*2*256, activation='relu')(decoder_input)
d1_dec = layers.Reshape((2, 2, 256))(d1_dec)

u1_dec = layers.Conv2DTranspose(nfilters*16, (2,2), strides=(2,2), padding='same')(d1_dec)
u1_dec = layers.concatenate([u1_dec, c4_dec])
u1_dec = layers.Dropout(0.1)(u1_dec)
c6_dec = layers.Conv2D(nfilters*16, kernel_size=(3,3),
               kernel_initializer='he_normal', padding='same')(u1_dec)
c6_dec = layers.BatchNormalization()(u1_dec)
c6_dec = layers.Activation('relu')(u1_dec)

u2_dec = layers.Conv2DTranspose(nfilters*8, (2,2), strides=(2,2), padding='same')(c6_dec)
u2_dec = layers.concatenate([u2_dec, c3_dec])
u2_dec = layers.Dropout(0.1)(u2_dec)
c7_dec = layers.Conv2D(nfilters*8, kernel_size=(3,3),
               kernel_initializer='he_normal', padding='same')(u2_dec)
c7_dec = layers.BatchNormalization()(u2_dec)
c7_dec = layers.Activation('relu')(u2_dec)

u3_dec = layers.Conv2DTranspose(nfilters*4, (2,2), strides=(2,2), padding='same')(c7_dec)
u3_dec = layers.concatenate([u3_dec, c2_dec])
u3_dec = layers.Dropout(0.1)(u3_dec)
c8_dec = layers.Conv2D(nfilters*4, kernel_size=(3,3),
               kernel_initializer='he_normal', padding='same')(u3_dec)
c8_dec = layers.BatchNormalization()(u3_dec)
c8_dec = layers.Activation('relu')(u3_dec)

u4_dec = layers.Conv2DTranspose(nfilters*2, (2,2), strides=(2,2), padding='same')(c8_dec)
u4_dec = layers.concatenate([u4_dec, c1_dec])
u4_dec = layers.Dropout(0.1)(u4_dec)
c9_dec = layers.Conv2D(nfilters*2, kernel_size=(3,3),
               kernel_initializer='he_normal', padding='same')(u4_dec)
c9_dec = layers.BatchNormalization()(u4_dec)
c9_dec = layers.Activation('relu')(u4_dec)


segmentation_output = layers.Conv2D(1, (1,1), activation='sigmoid', name='segmentation')(c9_dec)

## decoder model
decoder = keras.Model([decoder_input, c1_dec, c2_dec, c3_dec, c4_dec], segmentation_output, name='decoder')
decoder.summary()


## Modelo

In [15]:
class Unet(keras.Model):
    def __init__(self, unet_encoder, unet_decoder, **kwargs):
        super(Unet, self).__init__()
        self.decoder = unet_decoder
        self.encoder = unet_encoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(name="reconstruction_loss")
        
    @property
    def metrics(self):
        return [self.total_loss_tracker, self.reconstruction_loss_tracker]
    
    def train_step(self, data):
        image, mask, features, label = data
        with tf.GradientTape() as tape:
            embedding, activations = self.encoder(image, training=True)
            reconstruction = self.decoder([embedding] + activations, training=True)
            reconstruction_loss = tf.reduce_mean(keras.losses.binary_crossentropy(mask, reconstruction))
            total_loss = reconstruction_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        return {"total_loss": self.total_loss_tracker.result(), "reconstruction_loss": self.reconstruction_loss_tracker.result()}

In [ ]:
learning_rate = 0.001
epochs = 2000
batch = 8
patience = 15

unet = Unet(encoder,decoder)
opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
unet.compile(optimizer=opt)

unet.fit(train_dataset, epochs=epochs,
                batch_size=batch,
                         callbacks=[tf.keras.callbacks.EarlyStopping(monitor='total_loss', patience=30)])

In [ ]:
## save model
unet.encoder.save('Unet_Models/encoder_unet.h5')
unet.decoder.save('Unet_Models/decoder_unet.h5')